Набор данных

Nasdaq Data Link имеет большой набор данных для институциональных инвесторов, финтех-компаний и индивидуальных пользователей. 

Бесплатные и платные данные, доступ к которым можно получить через один из следующих API-интерфейсов Nasdaq Data Link:
- Streaming API для данных в реальном времени
- REST API для данных в реальном времени или с задержкой
- REST API для данных временных рядов
- REST API для данных таблиц

Взяты данные стоимости акций компании SAP c 03.09.2009 по 03.08.2019.

Проведете аналогичные действия для других наборов данных:
- [Rossmann Store Sales](https://www.kaggle.com/competitions/rossmann-store-sales/data)
- [Behavior of the urban traffic of the city of Sao Paulo in Brazil Data Set](http://archive.ics.uci.edu/ml/datasets/Behavior+of+the+urban+traffic+of+the+city+of+Sao+Paulo+in+Brazil)
- [ISTANBUL STOCK EXCHANGE Data Set](http://archive.ics.uci.edu/ml/datasets/ISTANBUL+STOCK+EXCHANGE)

In [ ]:
#подключаем библиотеки
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns



# Import metrics package from sklearn for statistical analysis
from sklearn import metrics
from sklearn.metrics import explained_variance_score


# Import package for splitting data set
from sklearn.model_selection import train_test_split

# Import package for linear model
from sklearn.linear_model import LinearRegression

from sklearn.svm import SVR

In [ ]:
url='https://raw.githubusercontent.com/yakushinav/mfc_ml/main/sap_stock.csv'
data=pd.read_csv(url,index_col='Date')

print(data.head())

In [ ]:
# типы данных столбцов
data.info()

In [ ]:
#описательная статистика
data.describe()

# Выберем множество значимых признаков

Мы используем дневную цену закрытия Close в качестве значения для прогнозирования, поэтому мы можем отбросить другие признаки.

Столбец «Close» имеет числовой тип данных
«Date» является столбцом индекса и содержит значения даты и времени.

In [ ]:
# Создадим новый набор данных
df = pd.DataFrame(data, columns=['Close'])

# Заменим индекс в форме даты на числовой индекс
df = df.reset_index()

# Проверим на пропущенные значения
print(df.isna().values.any())

print(df.head())

Анализ данных

Когда мы смотрим на изменение цены с течением времени на графике зависимости цены закрытия от времени, мы увидим, что цена постоянно увеличивается с течением времени, и мы также можем оценить, что тренд может быть линейным.

In [ ]:
# Подключаем пакет для работы с данным
import matplotlib.dates as mdates

years = mdates.YearLocator() # возьмем каждый год
yearsFmt = mdates.DateFormatter('%Y') # Формат отображения года

# Создаем график
fig, ax = plt.subplots()
ax.plot(df['Date'], df['Close'])

ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(yearsFmt)

plt.title('Динамика цены закрытия [2009 - 2019]', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Closing Stock Price in $', fontsize=14)
fig.autofmt_xdate()

plt.show()

# Линейная регрессия

Наши данные содержат только одну независимую переменную (X), которая представляет дату и зависимую переменную (Y), которую мы пытаемся предсказать, — это цена акций. Чтобы найти линию близкую к точкам данных,  мы можем использовать простую линейную регрессию.

Линия наилучшего соответствия может быть описана с помощью уравнения
$$
Y=\beta_0+\beta_1X
$$

где

  - Y прогнозируемое значение зависимой переменной
  - $\beta_0$ y-перечечение
  - $\beta_1$ наклон
  - X значение независимой переменной

Цель состоит в том, чтобы найти такие коэффициенты чтобы ошибка, которая представляет собой разницу между каждой точкой в наборе данных и соответствующим прогнозируемым значением, выводимым моделью была минимальна.

In [ ]:
# Разбиваем выборку
train, test = train_test_split(df, test_size=0.20)

In [ ]:

# Reshape index column to 2D array for .fit() method
X_train = np.array(train.index).reshape(-1, 1)
y_train = train['Close']

# Создаем модель машинного обучения
model = LinearRegression()
model.fit(X_train, y_train)

# Коэффициенты
print(model.coef_, model.intercept_)

In [ ]:


# График исходных данных и модели
plt.figure(1, figsize=(16,10))
plt.title('Linear Regression | Price vs Time')
plt.scatter(X_train, y_train, edgecolor='w', label='Actual Price')
plt.plot(X_train, model.predict(X_train), color='r', label='Predicted Price')
plt.xlabel('Integer Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

# Предсказание

In [ ]:
# Тестовые данные
X_test = np.array(test.index).reshape(-1, 1)
y_test = test['Close']

# Предсказание
y_pred = model.predict(X_test)

test['Prediction']=y_pred


In [ ]:
# График
fig, ax = plt.subplots()
test[0:25].plot(x='Date', y=['Close', 'Prediction'], kind='bar', ax=ax)

plt.title('Сравнение предсказанных и тестовых данных (первые 25 точек)', fontsize=16)

# 

# Set x label
plt.xlabel('Date', fontsize=14)

# Set y label
plt.ylabel('Stock Price in $', fontsize=14)

# Show plot
plt.show()

In [ ]:
# Добавим предсказанный тренд
df['Prediction'] = model.predict(np.array(df.index).reshape(-1, 1))
print(df.head())

# Рассчет ошибок MAE, MSE, RMSE
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2: ', metrics.r2_score(y_test, y_pred))
print(explained_variance_score(y_test, y_pred))


# Множественная линейная регрессия

In [ ]:
# Создадим новый набор данных
df = pd.DataFrame(data, columns=['Open','High','Low','Close'])

# Проверим на пропущенные значения
print(df.isna().values.any())

df=df.dropna()

# Проверим на пропущенные значения
print(df.isna().values.any())



In [ ]:
# Разделим выборку
train, test = train_test_split(df, test_size=0.20)

X=train[['Open','High','Low']] 
y=train['Close']

lm = LinearRegression()
lm.fit(X, y)

X_test=test[['Open','High','Low']] 
y_test=test['Close']

y_pred=lm.predict(X_test)

# Рассчет ошибок MAE, MSE, RMSE
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2: ', metrics.r2_score(y_test, y_pred))
print(explained_variance_score(y_test, y_pred))

# Метод опорных векторов

Метод опорных векторов (англ. SVM, support vector machine) — набор схожих алгоритмов обучения с учителем, использующихся для задач классификации и регрессионного анализа. Принадлежит семейству линейных классификаторов и может также рассматриваться как частный случай регуляризации по Тихонову. Особым свойством метода опорных векторов является непрерывное уменьшение эмпирической ошибки классификации и увеличение зазора, поэтому метод также известен как метод классификатора с максимальным зазором.

Радиальная базисная функция (РБФ) — функция из набора однотипных радиальных функций, используемых как функция активации в одном слое искусственной нейронной сети или как-либо ещё, в зависимости от контекста. Радиальная функция — это любая вещественная функция, значение которой зависит только от расстояния до начала координат.

In [ ]:
from sklearn.svm import SVR
# Создадим новый набор данных
df = pd.DataFrame(data, columns=['Close'])

# Заменим индекс в форме даты на числовой индекс
df = df.reset_index()

# Разделим выборку
train, test = train_test_split(df, test_size=0.20)

# Подготовим данные
X_train = np.array(train.index).reshape(-1, 1)
y_train = train['Close']

X_test=np.array(test.index).reshape(-1, 1)
y_test=test['Close']

svr_rbf = SVR(kernel="rbf", C=100, gamma=0.1, epsilon=0.1)
svr_lin = SVR(kernel="linear", C=100, gamma="auto")
svr_poly = SVR(kernel="poly", C=100, gamma="auto", degree=3, epsilon=0.1, coef0=1)

svr_rbf.fit(X_train, y_train)
#svr_lin.fit(X_train, y_train)
#svr_poly.fit(X_train, y_train)

# Train set graph
plt.figure(1, figsize=(16,10))
plt.title('RBF | Price vs Time')
plt.scatter(X_test, y_test, edgecolor='w', label='Actual Price')
plt.scatter(X_test, svr_rbf.predict(X_test), edgecolor='r', label='RBF')
#plt.scatter(X_test, svr_lin.predict(X_test), edgecolor='r', label='Linear')
#plt.scatter(X_test, svr_poly.predict(X_test), edgecolor='r', label='poly')

plt.xlabel('Integer Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()
